# Import module

In [2]:
from model import sefr_cut
from model.TH2IPA import TH2IPA
from model.IPA2ENG import IPA_matching
from neuspell import BertChecker
import re
import json

data folder is set to `e:\nlp\nlp-env\neuspell\neuspell\../data` script


# Load testing dataset

In [3]:
with open("./dict/clean_th_lyrics.json", encoding="utf8") as f:
    test_input = json.loads(f.read())

print(len(test_input))

FileNotFoundError: [Errno 2] No such file or directory: './dict/clean_th_lyrics.json'

In [ ]:
with open("./dict/clean_en_lyrics.json", encoding="utf8") as f:
    test_output = json.loads(f.read())

print(len(test_output))

# Initial variables

In [ ]:
th2ipa = TH2IPA()
checker = BertChecker()

sefr_cut.SEFR_CUT.load_model(engine='model')
checker.from_pretrained()

In [ ]:
tokenize_result = []
th2ipa_result = []
ipa2eng_result = []
corrector_result = []

# Pipeline

In [ ]:
def integration_pipeline(th_sent, th2ipa, checker):
    token_list = sefr_cut.tokenize(th_sent, k=40)[0]
    tokenize_result.append(token_list)
    ipa_list = th2ipa(token_list)
    th2ipa_result.append(ipa_list)
    eng_sent = ' '.join([IPA_matching(i)[0] for i in ipa_list]) 
    ipa2eng_result.append(eng_sent)
    correct_sent = checker.correct(eng_sent)
    correct_sent = re.sub(' \' ', '\'', correct_sent)
    corrector_result.append(correct_sent)
    return correct_sent

# Test